In [17]:
import pandas as pd
import seaborn as sns 
import numpy as np
import scipy as sp 
from  matplotlib import pyplot as plt
sns.set(font_scale=1.1, style="white",  #palette="viridis", 
        rc={'font.size': 16, 'figure.figsize': (7,4), 'axes.grid': True, 'lines.linewidth':2.0, 
            'grid.color': '.8', 'grid.linewidth': 0.5,})
np.set_printoptions(suppress=True)

In [18]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
!pip install GPy 
!pip install sklearn
import GPy as gpy
!git clone https://github.com/bestxolodec/DTR.git
!cd DTR; git pull 

    100% |################################| 829kB 1.3MB/s ta 0:00:01
    100% |################################| 81kB 5.4MB/s ta 0:00:011
  Running setup.py bdist_wheel for GPy ... - \ | / - \ | / - \ | / - \ | / done
  Stored in directory: /home/nbuser/.cache/pip/wheels/86/ed/02/3b671673c80d587be399021cef8ef4ad80cfe1bfcae16a1f9a
  Running setup.py bdist_wheel for paramz ... - \ done
  Stored in directory: /home/nbuser/.cache/pip/wheels/66/11/71/e91133fb7c1720f91f9cbd13e1d8d7f39a9359c9068664dc6f
Successfully built GPy paramz
You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
  Running setup.py bdist_wheel for sklearn ... - \ done
  Stored in directory: /home/nbuser/.cache/pip/wheels/d7/db/a3/1b8041ab0be63b5c96c503df8e757cf205c2848cf9ef55f85e
Successfully built sklearn
You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider

In [19]:
import sys
sys.path.append("/Users/ipaulo/yandexDisk/DIPLOMA/CODE/py-src/")

In [20]:
from itr import * 
# import rpy2.robjects as ro
# import rpy2
# ro.r.source("/Users/ipaulo/yandexDisk/DIPLOMA/CODE/src/clean_sources.R");

## Read data

In [255]:
X, Y = np.load("../data/sarcos/Xtr.npy"),  np.load("../data/sarcos/Ytr.npy")
rng = np.random.RandomState(777)
n_train, n_test, n_val = 2000, 1000, 100
shuffle_idxs = np.arange(X.shape[0])
np.random.shuffle(shuffle_idxs) 
n_restarts = 1
Xtr, Xts, Xvl, _ = np.split(X[shuffle_idxs, :], np.cumsum([n_train, n_test, n_val]))
Ytr, Yts, Yvl, _ = np.split(Y[shuffle_idxs, :], np.cumsum([n_train, n_test, n_val]))

###  Simple miltioutput GP 

In [256]:
def get_mogp_score(preds, test):
    """return mean reward â aka mean squared euclidian distance"""
    if isinstance(test, dict):
        return np.square(test["optimal.treatment"] - preds).sum(axis=1).mean()
    else:
        return np.square(Yts - preds).sum(axis=1).mean()
    
def pred_value(preds, test):
    return - get_mogp_score(preds, test) 

### Baseline regression model

In [258]:
from sklearn.linear_model import LinearRegression
from sklearn.base import clone

class BaselineModel(object):
    def __init__(self, *args, **kwargs): 
        self.m = LinearRegression(*args, **kwargs)

    def fit(self, X, Y):
        self.models = [clone(self.m).fit(X, Y[:, i]) for i in range(Y.shape[1])]
        return self
        
    def predict(self, Xts):
        return np.hstack([m.predict(Xts).reshape(-1,1) for m in self.models])
    
    def get_score(self, Xts, Yts):
        return np.square(Yts - self.predict(Xts)).sum(axis=1).mean()
    
    def get_rewards_for(self, Xts, Yts):  
        return - np.log(np.square(Yts - self.predict(Xts)).sum(axis=1).reshape(-1,1))
        return - np.square(Yts - self.predict(Xts)).sum(axis=1).reshape(-1,1)
        # TODO: normal shaped alternative 
    
    
    def build_dataset(self, Xtr, Ytr):
        df = pd.DataFrame(np.hstack([Xtr, self.predict(Xtr), self.get_rewards_for(Xtr, Ytr)]))
        df.columns = np.hstack([["C{}".format(i) for i in range(Xtr.shape[1])], 
                                ["A{}".format(i) for i in range(Ytr.shape[1])],
                                ["R"]]) 
        return df

    def get_dataset_in_dict(self, Xtr, Ytr):
        return {"covariates": Xtr, "treatment": self.predict(Xtr), 
                "reward": self.get_rewards_for(Xtr, Ytr), "optimal.treatment": Ytr}
    


In [259]:
bm = BaselineModel(n_jobs=-1).fit(X, Y)
train = bm.get_dataset_in_dict(Xtr, Ytr)
test = bm.get_dataset_in_dict(Xts, Yts)

In [253]:
%%time
s_factors_percs = np.arange(.5,1, .01)
s_factors = sp.stats.norm.ppf(s_factors_percs) # 50 factors evenly splitted
granularity = 50 
fit_params = {"mean_fn": False, "n_restarts": 1, "verbose":False, 
              "robust":True, "normalize": False}
preds, v, m = fit_and_predict(train, test, granularity, s_factors, pred_value, fit_params) 

KeyboardInterrupt caught, calling on_optimization_end() to round things up


KeyboardInterrupt: 

In [254]:
v

itr.py is imported!


array(-18753429.767093055)

In [134]:
%%time
s_factors_percs = np.arange(.5,1, .01)
s_factors = sp.stats.norm.ppf(s_factors_percs) # 50 factors evenly splitted
granularity = 50 
fit_params = {"mean_fn": False, "n_restarts": 1, "verbose":False, 
              "robust":True, "normalize": False}
preds, v, m = fit_and_predict(train, test, granularity, s_factors, pred_value, fit_params) 

CPU times: user 2h 1min 42s, sys: 22min 34s, total: 2h 24min 16s
Wall time: 3h 34min 26s


In [148]:
v

array(-180930759.62113008)

###  bred


In [187]:
[i[0] ** 2 for i in data]

[5329, 196, 2916, 2916, 9604, 6241, 36, 5625, 7396, 5329]

In [218]:
def worker(c, m=m, x0=x0):
    def f(a): return m.predict(np.hstack([c, a]).reshape(1,-1))[0]  # mean
    return go_amp.AMPGO(f, x0)[0]

In [219]:
import pickle

In [222]:
pickle.dumps()

b'\x80\x03cnumpy.core.multiarray\n_reconstruct\nq\x00cnumpy\nndarray\nq\x01K\x00\x85q\x02C\x01bq\x03\x87q\x04Rq\x05(K\x01K\x1c\x85q\x06cnumpy\ndtype\nq\x07X\x02\x00\x00\x00f8q\x08K\x00K\x01\x87q\tRq\n(K\x03X\x01\x00\x00\x00<q\x0bNNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00tq\x0cb\x89C\xe0re\xab\xd7\x85\xe5\xe0?A\xa1Z\t6%\xe5?0XP\xc7\x05\x88\xe2?\xa45\x8f(\xc2?\xe3?\xc6\x1c8\n\xbd\x1d\xd7?t{\xdf\xf6\xab\xf2\xe6?\x1a\x9a[\x98\xbb\xe0\xdc?\x082\x80\xbd\xc4~\xe2?\x15\x9bg\x1a\xb8\x9f\xe0?8\xae\x1e\\\x85q\xd9?\xb3\xdfF\xa8\x18a\xe2?=\xd7J\xd7t\xa2\xe1?Q\xd7M\xf6}V\xdf?u"\x8e\xa6.l\xe0?/\x84\xfa\xd8\xaf\xbc\xde?\xa8\xb5V9\xf1\xcb\xd9?w\xc7\xd2RJ4\xd9?d\xfd\xa3 \xae\xbe\xd8?\xc0\x1fI5\xc8!\xdd?8=\xfbta\xd4\xd6?\xedc\xb7\x9d\x8e\xf3\xe2?\xa0\xa7\x99\xa9\xbe\x85\xda?\x04G\x02\xb2\x125\xe3?\xaezQ\x99]I\xd5?\x89\xf3-e\xb4\xd5\xde?\xae\xe9\xc6\xf4\xdbW\xd5?\xc1\xae\xeaz\xa9\x84\xdb?\xfa\xe0\xa8\x05\xad\x0e\xdf?q\rtq\x0eb.'

In [227]:
from itertools import zip_longest

In [ ]:
[[C[i], m, x0] for i in range(C.shape[0])]

In [230]:
list(zip_longest([1,2,3], [1], [2]))

[(1, 1, 2), (2, None, None), (3, None, None)]

In [216]:
from time import sleep

def worker(d):
    v,t  = d
    print("started ", v, " to sleep ", t)
    sleep(t)
    print("exited ", v)
    return v


N = 10 
data = list(zip(np.arange(N), np.hstack([30, np.random.randint(10, size=N-1)] )))
data = list(zip(np.arange(N), C[:N, :]))

from multiprocessing import  Pool
p = Pool(8)
p.map(worker, data)

started  2  to sleep  [ 0.4513116   0.81111295  0.4356255   0.43192753  0.32760303  0.65282232
  0.60608696  0.45676337  0.29119162  0.51675701  0.69031367  0.64830508
  0.4802662   0.4017254   0.5106815   0.40970824  0.60372622  0.48795719
  0.62452502  0.45807592  0.3074763 ]
started  7  to sleep  [ 0.49335041  0.73468421  0.53026711  0.61549014  0.39453505  0.65103641
  0.34811338  0.51596268  0.60751629  0.47913637  0.30880295  0.4098496
  0.57008805  0.84796893  0.45451339  0.55139105  0.3716837   0.22998429
  0.4576217   0.51562902  0.71911012]started  9  to sleep  [ 0.17683132  0.24016431  0.59035228  0.88197919  0.90519165  0.363659
  0.14085652  0.484141    0.09912653  0.75151667  0.66478993  0.92014697
  0.5325043   0.41859939  0.46386107  0.47595322  0.41877831  0.30737516
  0.48906584  0.52922331  0.7233067 ]



TypeError: only length-1 arrays can be converted to Python scalars

started  4  to sleep  [ 0.90257607  0.32690132  0.85913429  0.11690552  0.23507331  0.77382864
  0.22370583  0.44411319  0.48914729  0.51182999  0.37273747  0.56798591
  0.54705205  0.443236    0.30673056  0.6715423   0.43340019  0.54268505
  0.47685411  0.45167176  0.67725741]


In [196]:
dir(p)

['Process',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cache',
 '_ctx',
 '_get_tasks',
 '_handle_results',
 '_handle_tasks',
 '_handle_workers',
 '_help_stuff_finish',
 '_initargs',
 '_initializer',
 '_inqueue',
 '_join_exited_workers',
 '_maintain_pool',
 '_map_async',
 '_maxtasksperchild',
 '_outqueue',
 '_pool',
 '_processes',
 '_quick_get',
 '_quick_put',
 '_repopulate_pool',
 '_result_handler',
 '_setup_queues',
 '_state',
 '_task_handler',
 '_taskqueue',
 '_terminate',
 '_terminate_pool',
 '_worker_handler',
 '_wrap_exception',
 'apply',
 'apply_async',
 'close',
 'imap',
 'imap_unordered',
 'join',
 'map',
 'map_async',
 'starmap',
 'starmap_async',


###  Simple miltioutput GP 

In [ ]:
%%time
k = gpy.kern.RBF(21)
# m = gpy.models.GPRegression(Xtr, Ytr, kernel=k) 
m = gpy.models.SparseGPRegression(Xtr, Ytr, kernel=k, num_inducing=2000) 
m.optimize_restarts(num_restarts=n_restarts, verbose=False, robust=True);

In [ ]:
get_mogp_score(m.predict(Xts)[0], Yts) # 2k inducing, 10k train & 10k test

In [10]:
get_mogp_score(m.predict(Xts)[0], Yts) # 1k inducing, 10k train & 10k test

20.035417479278109

In [11]:
get_mogp_score(m.predict(Xts)[0], Yts) # 7k train & 10k test

21.676514221977374

In [9]:
get_mogp_score(m.predict(Xts)[0], Yts) # 5k train & 10k test

26.705097341917799

In [39]:
get_mogp_score(m.predict(Xts)[0], Yts) # 1k train & 10k test

43.73352777260969

In [30]:
get_mogp_score(m.predict(Xts)[0], Yts) # 1k inducing on 10k train & 10k test

19.895384276908555

In [26]:
get_mogp_score(m.predict(Xts)[0], Yts) # 500 inducing on 5k train & 5k test

28.481604519136983

In [24]:
get_mogp_score(m.predict(Xts)[0], Yts)

101.99679684192976

### Coregionalized regression

In [ ]:
kernel = GPy.util.multioutput.ICM(input_dim=1,num_outputs=2,kernel=GPy.kern.Bias(input_dim=1))
m = GPy.models.GPCoregionalizedRegression(X_list=[X1,X2],Y_list=[Y1,Y2],kernel=kernel)
m['.*bias.var'].constrain_fixed(1) #B.kappa now encodes the variance.
m['.*W'].constrain_fixed(0)
m.optimize()

In [ ]:
K = GPy.kern.Matern32(1)
icm = GPy.util.multioutput.ICM(input_dim=1,num_outputs=2,kernel=K)

m = GPy.models.GPCoregionalizedRegression([X1,X2],[Y1,Y2],kernel=icm)
m['.*Mat32.var'].constrain_fixed(1.) #For this kernel, B.kappa encodes the variance now.
m.optimize()

In [111]:
from itertools import  repeat

In [ ]:
n_output = 7
icm = gpy.util.multioutput.ICM(input_dim=21,num_outputs=n_output,kernel=gpy.kern.RBF(21))
m = gpy.models.GPCoregionalizedRegression(list(repeat(Xtr, n_output)), np.split(Ytr, 7, axis=1), kernel=icm)
m.optimize()
# prediction data should include number of component
m.predict(list(repeat(Xtr, n_output))) # this will fail